### 第九章 分类数据

In [215]:
import numpy as np
import pandas as pd

#### 一、cat对象
##### 1.cat对象的属性
在`pandas`中提供了`category`类型，使用户能够处理分类类型的变量，将一个普通序列转换成分类变量可以使用`astype`方法.

In [216]:
df = pd.read_csv('../pandas_basis/learn_pandas.csv',usecols=['Grade','Name','Gender','Height','Weight'])

In [217]:
s = df.Grade.astype('category')
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']

在一个分类类型的`Series`中定义了`cat`对象，它和上一章中介绍的`str`对象类似，定义了一些属性和方法来进行分类类别的操作。

In [218]:
s.cat

对于一个具体的分类，有两个组成部分，其一为类别的本身，它以`Index`类型存储，其二为是否有序，它们都可以通过`cat`的属性被访问:

In [219]:
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')

In [220]:
s.cat.ordered

False

另外，每一个序列的类别会被赋予唯一的整数编号，它们的编号取决于`cat.categories`中的顺序，该属性可以通过`codes`访问:

In [221]:
s.cat.codes.head()

0    0
1    0
2    2
3    3
4    3
dtype: int8

##### 2.类别的增加、删除和修改
通过对`cat`对象的`categories`属性能够完成对类别的查询，那么应该如何进行“增改查删”的其他三个操作呢？         
**类别不得直接修改**         
在第三章中曾提到，索引`Index`类型是无法用`index_obj[0] = item`来修改的，而`categories`被存储在`Index`中，因此`pandas`在`cat`属性上定义了若干方法来达到相同的目的。

首先，对于类别的增加可以使用`add_categories`：

In [222]:
s = s.cat.add_categories('Graduate') # 增加一个毕业生类别
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

若要删除某一个类别可以使用`remove_categories`，同时所有原来序列中的该类会被设置为缺失。例如，删除大一的类别：

In [223]:
s = s.cat.remove_categories('Freshman')  # 删除大一类别
s.cat.categories

Index(['Graduate', 'Junior', 'Senior', 'Sophomore'], dtype='object')

In [224]:
s.head()

0          NaN
1          NaN
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Graduate', 'Junior', 'Senior', 'Sophomore']

此外可以使用`set_categories`直接设置序列的新类别，原来的类别中如果存在元素不属于新类别，那么会被设置为缺失。

In [225]:
s = s.cat.set_categories(['Sophomore', 'PhD']) # 新类别为大二学生和博士
s.cat.categories

Index(['Sophomore', 'PhD'], dtype='object')

In [226]:
s.head()

0          NaN
1          NaN
2          NaN
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (2, object): ['Sophomore', 'PhD']

如果想要删除未出现在序列中的类别，可以使用`remove_unused_categories`来实现:

In [227]:
s = s.cat.remove_unused_categories() # 删除未出现的类别
s.cat.categories

Index(['Sophomore'], dtype='object')

最后，“增改查删”中还剩下修改的操作，这可以通过`rename_categories`方法来完成，同时需要注意的是，这个方法会对原序列的对应值也进行相应修改。例如，现在把`Sophomore`改为中文的`本科二年级学生`：

In [228]:
s = s.cat.rename_categories({'Sophomore':'本科二年级学生'})
s.head()

0        NaN
1        NaN
2        NaN
3    本科二年级学生
4    本科二年级学生
Name: Grade, dtype: category
Categories (1, object): ['本科二年级学生']

#### 二、有序分类
##### 1.序的建立
有序类别和无序类别可以通过`as_unordered`和`reorder_categories`互相转化，需要注意的是后者传入的参数必须是由当前序列的无序类别构成的列表，不能够增加新的类别，也不能缺少原来的类别，并且必须指定参数`ordered=True`,否则方法无效。例如，对年级高低进行相对大小的类别划分，然后再恢复无序状态:

In [229]:
s = df.Grade.astype('category')
s = s.cat.reorder_categories(['Freshman','Sophomore','Junior','Senior'], ordered=True)
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']

In [230]:
s.cat.as_unordered().head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Sophomore', 'Junior', 'Senior']

**类别不得直接修改**          
如果不想指定`ordered=True`参数，那么可以先用`s.cat.as_ordered()`转化为有序类别，再利用`reorder_categories`进行具体的相对大小调整。

##### 2.排序和比较
在第二章中，曾提到了字符串和数值类型序列的排序，此时就要说明分类变量的排序：只需把列的类型修改为`category`后，再赋予相应的大小关系，就能正常地使用`sort_index`和`sort_values`，例如，对年级进行排序:

In [231]:
df.Grade = df.Grade.astype('category')
df.Grade = df.Grade.cat.reorder_categories(['Freshman','Sophomore','Junior','Senior'], ordered=True)
df.sort_values('Grade').head() # 值排序

,Grade,Name,Gender,Height,Weight
0,Freshman,Gaopeng Yang,Female,158.9,46.0
105,Freshman,Qiang Shi,Female,164.5,52.0
96,Freshman,Changmei Feng,Female,163.8,56.0
88,Freshman,Xiaopeng Han,Female,164.1,53.0
81,Freshman,Yanli Zhang,Female,165.1,52.0


In [232]:
df.set_index('Grade').sort_index().head() # 索引排序

,Name,Gender,Height,Weight
Grade,,,,
Freshman,Gaopeng Yang,Female,158.9,46.0
Freshman,Qiang Shi,Female,164.5,52.0
Freshman,Changmei Feng,Female,163.8,56.0
Freshman,Xiaopeng Han,Female,164.1,53.0
Freshman,Yanli Zhang,Female,165.1,52.0


由于序的建立，因此就可以进行比较操作。分类变量的比较操作分为两类，第一种是==或!=关系的比较，比较的对象可以是标量或者是同长度的`Series`(或`list`),第二种是`>,>=,<,<=`四类大小关系的比较，比较的对象和第一种类似，但是所有参与比较的元素必须属于原序列的`categories`,同时要和原序列具有相同的索引.

In [233]:
res1 = df.Grade == 'Sophomore'
res1.head()

0    False
1    False
2    False
3     True
4     True
Name: Grade, dtype: bool

In [234]:
res2 = df.Grade == ['PhD'] * df.shape[0]
res2.head()

0    False
1    False
2    False
3    False
4    False
Name: Grade, dtype: bool

In [235]:
res3 = df.Grade <= 'Sophomore'
res3.head()

0     True
1     True
2    False
3     True
4     True
Name: Grade, dtype: bool

In [236]:
res4 = df.Grade <= df.Grade.sample(frac=1).reset_index(drop=True) 
# 关键步骤。因为 Pandas 进行运算时会自动对齐索引（Align by Index）
# 如果不重置索引，左侧的 df.Grade 和右侧打乱后的 Series 会根据原始索引重新匹配回去，导致实际上是自己跟自己比（结果永远是 True）。
res4.head()

0     True
1     True
2    False
3     True
4     True
Name: Grade, dtype: bool

#### 三、区间类别
##### 1.利用cut和qcut进行区间构造
区间是一种特殊的类别，在实际数据分析中，区间序列往往是通过`cut`和`qcut`方法进行构造的，这两个函数能够把原序列的数值特征进行装箱，即用区间位置来代替原来的具体数值。

首先介绍`cut`的常见用法:              
其中，最重要的参数是`bins`，如果传入整数`n`，则代表把整个传入数组的按照最大和最小值等间距地分为n段。由于区间默认是左开右闭，需要在调整时把最小值包含进去，在`pandas`中的解决方案是在值最小的区间左端点再减去`0.001*(max-min)`,因此，如果对序列[1,2]划分为2个箱子时，第一个箱子的范围`(0.999,1.5]`,第二个箱子的范围是`(1.5,2]`。如果需要指定左闭右开时，需要把`right`参数设置为`False`,相应的区间调整方法是在值最大的区间右端点再加上`0.001*(max-min)`。

In [237]:
s = pd.Series([1,2])
pd.cut(s,bins = 2)

0    (0.999, 1.5]
1      (1.5, 2.0]
dtype: category
Categories (2, interval[float64, right]): [(0.999, 1.5] < (1.5, 2.0]]

In [238]:
pd.cut(s,bins = 2,right = False)

0      [1.0, 1.5)
1    [1.5, 2.001)
dtype: category
Categories (2, interval[float64, left]): [[1.0, 1.5) < [1.5, 2.001)]

`bins`的另一个常见用法是指定区间分割点的列表(使用`np.infty`)可以表示无穷大:

In [239]:
pd.cut(s, bins = [-np.inf,1.2,1.8,2.2,np.inf])

0    (-inf, 1.2]
1     (1.8, 2.2]
dtype: category
Categories (4, interval[float64, right]): [(-inf, 1.2] < (1.2, 1.8] < (1.8, 2.2] < (2.2, inf]]

另外两个常用参数为`labels`和`retbins`,分别代表了区间的名字和是否返回分割点(默认不返回):

In [240]:
s = pd.Series([1,2])
res = pd.cut(s, bins = 2, labels = ['small','big'], retbins = True)
res[0]

0    small
1      big
dtype: category
Categories (2, object): ['small' < 'big']

In [241]:
res[1] # 该元素为返回的分割点

array([0.999, 1.5  , 2.   ])

从用法上来说，`qcut`和`cut`几乎没有差别，只是把`bins`参数变成的`q`参数,`qcut`中的`q`是指`quantile`.这里的`q`为整数`n`时，指按照`n`等分位数把数据分箱，还可以传入浮点列表指代相应的分位数分割点。

In [242]:
s = df.Weight
pd.qcut(s, q = 3).head()

0    (33.999, 48.0]
1      (55.0, 89.0]
2      (55.0, 89.0]
3    (33.999, 48.0]
4      (55.0, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64, right]): [(33.999, 48.0] < (48.0, 55.0] < (55.0, 89.0]]

In [243]:
pd.qcut(s,q=[0,0.2,0.8,1]).head()

0      (44.0, 69.4]
1      (69.4, 89.0]
2      (69.4, 89.0]
3    (33.999, 44.0]
4      (69.4, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64, right]): [(33.999, 44.0] < (44.0, 69.4] < (69.4, 89.0]]

##### 2.一般区间的构造
对于某一个具体的区间而言，其具备三个要素，即左端点、右端点和端点的开闭状态，其中开闭状态可以指定`right,left,both,neither`中的一类:

In [244]:
my_interval = pd.Interval(0,1, 'right')
my_interval

Interval(0, 1, closed='right')

其属性包含了`mid,length,right,left,closed`，分别表示中点、长度、右端点、左端点和开闭状态。         
使用`in`可以判断元素是否属于区间:

In [245]:
0.5 in my_interval

True

使用`overlaps`可以判断两个区间是否有交集:

In [246]:
my_interval_2 = pd.Interval(0.5,1.5, 'left')
my_interval.overlaps(my_interval_2)

True

一般而言，`pd.IntervalIndex`对象有四类方法生成，分别是`from_breaks,from_arrays,from_tuples,interval_range`,它们分别应用于不同的情况:

`from_breaks`的功能类似于`cut`或`qcut`函数，只不过后两个是通过计算得到的分割点，而前者是直接传入自定义的分割点:

In [247]:
pd.IntervalIndex.from_breaks([1,3,6,10], closed = 'both')

IntervalIndex([[1, 3], [3, 6], [6, 10]], dtype='interval[int64, both]')

`from_arrays`是分别传入左端点和右端点的列表，适用于有交集并且知道起点和终点的情况:

In [248]:
pd.IntervalIndex.from_arrays(left = [1,3,6,10],
                             right = [5,4,9,11],
                             closed = 'neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)], dtype='interval[int64, neither]')

`from_tuples`传入的是起点和终点元组构成的列表:

In [249]:
pd.IntervalIndex.from_tuples([(1,5),(3,4),(6,9),(10,11)],closed = 'neither')

IntervalIndex([(1, 5), (3, 4), (6, 9), (10, 11)], dtype='interval[int64, neither]')

一个等差的区间序列由起点、终点、区间个数和区间长度决定，其中三个量确定的情况下，剩下一个量就确定了，`interval_range`中的`start,end,periods,freq`参数就对应了这四个量，从而就能构造出相应的区间:

In [250]:
pd.interval_range(start=1,end=5,periods = 8)

IntervalIndex([(1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5],
               (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]],
              dtype='interval[float64, right]')

In [251]:
pd.interval_range(end = 5, periods = 8, freq = 0.5)

IntervalIndex([(1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5],
               (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]],
              dtype='interval[float64, right]')

除此之外，如果直接使用`pd.IntervalIndex([...],closed=...)`,把`Interval`类型的列表组成传入其中转为区间索引，那么所有的区间会被强制转为指定的`closed`类型，因为`pd.IntervalIndex`只允许存放同一种开闭区间的`Interval`对象。

In [252]:
my_interval

Interval(0, 1, closed='right')

In [253]:
my_interval_2

Interval(0.5, 1.5, closed='left')

In [254]:
pd.IntervalIndex([my_interval, my_interval_2], closed = 'left')

IntervalIndex([[0.0, 1.0), [0.5, 1.5)], dtype='interval[float64, left]')

##### 3.区间的属性与方法
`IntervalIndex`上也定义了一些有用的属性和方法。同时，如果想要具体利用`cut`或者`qcut`的结果进行分析，那么需要先将其转为该种索引类型:

In [255]:
id_interval = pd.IntervalIndex(pd.cut(s, bins = 3))

与单个`Interval`类型相似，`IntervalIndex`有若干常用属性:`left, right, mid, length`,分别表示左右端点、两端点均值和区间长度。

In [256]:
id_demo = id_interval[:5]
id_demo

IntervalIndex([(33.945, 52.333], (52.333, 70.667],   (70.667, 89.0],
               (33.945, 52.333],   (70.667, 89.0]],
              dtype='interval[float64, right]', name='Weight')

In [257]:
id_demo.left

Index([33.945, 52.333, 70.667, 33.945, 70.667], dtype='float64')

In [258]:
id_demo.right

Index([52.333, 70.667, 89.0, 52.333, 89.0], dtype='float64')

In [259]:
id_demo.mid

Index([43.138999999999996, 61.5, 79.8335, 43.138999999999996, 79.8335], dtype='float64')

In [260]:
id_demo.length

Index([18.387999999999998, 18.334000000000003, 18.333, 18.387999999999998,
       18.333],
      dtype='float64')

`IntervalIndex`还有两个常用方法，包括`contains`和`overlaps`，分别指逐个判断每个区间是否包含某元素，以及是否和一个`pd.Interval`对象有交集。

In [261]:
id_demo.contains(4)

array([False, False, False, False, False])

In [262]:
id_demo.overlaps(pd.Interval(40,60))

array([ True,  True, False,  True, False])

#### 四、练习
##### Ex1: 统计未出现的类别
在第五章中介绍了`crosstab`函数，在默认参数下它能够对两个列的组合出现的频数进行统计汇总:

In [263]:
df = pd.DataFrame({
    'A': ['a','b','c','a'],
    'B': ['cat','cat','dog','cat']
})
pd.crosstab(df.A, df.B)

B,cat,dog
A,,
a,2,0
b,1,0
c,0,1


但事实上有些列存储的是分类变量，列中并不一定包含所有的类别，此时如果想要对这些未出现的类别在`crosstab`结果中也进行汇总，则可以指定`dropna`参数为`False`:

In [264]:
df.B = df.B.astype('category').cat.add_categories('sheep')
pd.crosstab(df.A, df.B, dropna = False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0


请实现带有一个`dropna`参数的`my_crosstab`函数来完成上面的功能。

In [311]:
def my_crosstab(s1, s2, dropna=True):
    idx1 = (s1.cat.categories if s1.dtype.name == 'category' and not dropna else s1.unique())
    idx2 = (s2.cat.categories if s2.dtype.name == 'category' and not dropna else s2.unique())
    res = pd.DataFrame(np.zeros((idx1.shape[0], idx2.shape[0])), index=idx1, columns=idx2)
    for i, j in zip(s1, s2):
        res.at[i, j] += 1
    res = res.rename_axis(index=s1.name, columns=s2.name).astype('Int64')
    return res
df = pd.DataFrame({'A':['a','b','c','a'], 'B':['cat','cat','dog','cat']})
df.B = df.B.astype('category').cat.add_categories('sheep')
my_crosstab(df.A, df.B)
my_crosstab(df.A, df.B, dropna=False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0


##### Ex2：钻石数据集
现有一份关于钻石的数据集，其中`carat,cut,clarity,price`分别表示克拉重量、切割质量、纯净度和价格，样例如下:

In [265]:
df = pd.read_csv('diamonds.csv')
df.head()

,carat,cut,clarity,price
0,0.23,Ideal,SI2,326
1,0.21,Premium,SI1,326
2,0.23,Good,VS1,327
3,0.29,Premium,VS2,334
4,0.31,Good,SI2,335


In [275]:
# 1.分别对df.cut在object类型和category类型下使用nunique函数，比较它们的性能
s_obj, s_cat = df.cut, df.cut.astype('category')
%timeit -n 30 s_obj.nunique()

1.52 ms ± 120 μs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [276]:
%timeit -n 30 s_cat.nunique()

192 μs ± 18.6 μs per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [277]:
# 2.钻石的切割质量可以分为五个等级，由次到好分别是Fair, Good, Very Good, Premium, Ideal,
# 纯净度有八个等级，由差到好分别是I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF
# 请对切割质量按照由好到次的顺序排序，相同切割质量的钻石，按照纯净度进行由次到好的排序
cut_categories = ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
df.cut = df.cut.astype('category').cat.reorder_categories(cut_categories, ordered = True)
df.clarity = df.clarity.astype('category').cat.reorder_categories(clarity_categories, ordered = True)
df_sorted = df.sort_values(by = ['cut','clarity'])
df_sorted.head()

,carat,cut,clarity,price
315,0.96,Ideal,I1,2801
535,0.96,Ideal,I1,2826
551,0.97,Ideal,I1,2830
653,1.01,Ideal,I1,2844
718,0.97,Ideal,I1,2856


In [278]:
# 3.分别采用两种不同的方法，把cut,clarity这两列按照由好到次的顺序，映射到从0到n-1的整数，其中n表示类别的个数
cut_map = {cat:idx for idx, cat in enumerate(cut_categories)}
clarity_map = {cat:idx for idx, cat in enumerate(clarity_categories[::-1])}
df.cut.map(cut_map)

0        0
1        1
2        3
3        1
4        3
        ..
53935    0
53936    3
53937    2
53938    1
53939    0
Name: cut, Length: 53940, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [279]:
df.clarity.map(clarity_map)

0        6
1        5
2        3
3        4
4        6
        ..
53935    5
53936    5
53937    5
53938    6
53939    6
Name: clarity, Length: 53940, dtype: category
Categories (8, int64): [7 < 6 < 5 < 4 < 3 < 2 < 1 < 0]

In [282]:
df.clarity.cat.reorder_categories(clarity_categories[::-1], ordered = True).cat.codes

0        6
1        5
2        3
3        4
4        6
        ..
53935    5
53936    5
53937    5
53938    6
53939    6
Length: 53940, dtype: int8

In [283]:
df.cut.cat.codes

0        0
1        1
2        3
3        1
4        3
        ..
53935    0
53936    3
53937    2
53938    1
53939    0
Length: 53940, dtype: int8

In [286]:
# 4.对每克拉的价格分别按照分位数(q=[0.2,0.4,0.6,0.8]) 与 [1000, 3500, 5500, 18000]割点进行分箱
# 得到五个类别Very Low, Low, Mid, High, Very High,并把按这两种分箱方法得到的category序列依次添加到原表中.
price_per_carat = df.price / df.carat
q = [0,0.2,0.4,0.6,0.8,1]
price_bins_q = pd.qcut(price_per_carat, q = q, 
                        labels = ['Very Low', 'Low', 'Mid', 'High', 'Very High'])
df['price_per_carat_qbin'] = price_bins_q
bins = [ -np.inf, 1000, 3500, 5500, 18000, np.inf]
price_bins = pd.cut(price_per_carat, bins = bins,
                    labels = ['Very Low', 'Low', 'Mid', 'High', 'Very High'])
df['price_per_carat_bin'] = price_bins
df.head()

,carat,cut,clarity,price,price_per_carat_qbin,price_per_carat_bin
0,0.23,Ideal,SI2,326,Very Low,Low
1,0.21,Premium,SI1,326,Very Low,Low
2,0.23,Good,VS1,327,Very Low,Low
3,0.29,Premium,VS2,334,Very Low,Low
4,0.31,Good,SI2,335,Very Low,Low


In [291]:
# 5.第4问中按照整数分箱得到的序列中，是否出现了所有的类别？如果存在没有出现的类别请把该类别删除
df.price_per_carat_bin.cat.remove_unused_categories().cat.categories

Index(['Low', 'Mid', 'High'], dtype='object')

In [309]:
# 6.对第4问中按照分位数分箱得到的序列，求每个样本对应所在区间的左右端点值和长度。
intervals = pd.IntervalIndex(pd.qcut(price_per_carat, q = q))
intervals.left
intervals.right
intervals.length

Index([         1243.838,          1243.838,          1243.838,
                1243.838,          1243.838,          1243.838,
                1243.838,          1243.838,          1243.838,
                1243.838,
       ...
       958.3899999999999, 958.3899999999999, 958.3899999999999,
       958.3899999999999, 958.3899999999999, 958.3899999999999,
       958.3899999999999, 958.3899999999999, 958.3899999999999,
       958.3899999999999],
      dtype='float64', length=53940)